Try out the gnn model


In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
adata_rna = sc.read_h5ad("/lustre/groups/ml01/workspace/eirini.giannakoulia/datasets/V11L12-038/V11L12-038_A1/V11L12-038_A1.RNA_MOSCOT_paired_hvg.h5ad")
adata_rna

In [ ]:
adata_msi = sc.read_h5ad("/lustre/groups/ml01/workspace/eirini.giannakoulia/datasets/V11L12-038/V11L12-038_A1/V11L12-038_A1.MSI_MOSCOT_paired_hvg.h5ad")
adata_msi

In [ ]:
adata_rna_train = adata_rna[adata_rna.obs["split"] == "train"]
adata_rna_test = adata_rna[adata_rna.obs["split"] == "test"]
adata_msi_train = adata_msi[adata_msi.obs["split"] == "train"]
adata_msi_test = adata_msi[adata_msi.obs["split"] == "test"]

In [ ]:
coords_rna_train = adata_rna_train.obsm["spatial_warp"]
coords_rna_test = adata_rna_test.obsm["spatial_warp"]

In [ ]:
class MultiLayerGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(MultiLayerGCN, self).__init__()
        
        # Define GCN layers
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim))  # First layer
        
        # Add hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GCNConv(hidden_dim, hidden_dim))  # Middle layers
        
        self.output_layer = GCNConv(hidden_dim, output_dim)  # Output layer
        
        self.dropout = dropout

    def forward(self, x, edge_index):
        # Pass data through each GCN layer with ReLU activation
        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Final GCN layer (output layer)
        x = self.output_layer(x, edge_index)
        
        return x  

In [ ]:
# Define graph structure using k-nearest neighbors
k = 15  # Number of neighbors for KNN
knn = NearestNeighbors(n_neighbors=k).fit(coords_rna_train)
distances, indices = knn.kneighbors(coords_rna_train)

edges = []
for i, neighbors in enumerate(indices):
    for neighbor in neighbors:
        if i != neighbor:  # Avoid self-loops
            edges.append([i, neighbor])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # Shape [2, num_edges]

# Initialize GCN model
input_dim = matching_rna_train.shape[1]  # Number of input features
hidden_dim = 256  # Hidden layer size
output_dim = matching_msi_train.shape[1]  # Number of output features
num_layers = 3  # Number of GCN layers
dropout = 0.3  # Dropout rate

model = MultiLayerGCN(input_dim, hidden_dim, output_dim, num_layers=num_layers, dropout=dropout)

# Prepare PyTorch data
train_data = Data(x=X_tensor, edge_index=edge_index)

# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
num_epochs = 2000

# Train GCN model
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = criterion(out, y_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Convert test data to PyTorch tensors
X_tensor_test = torch.tensor(matching_rna_test, dtype=torch.float32)
y_tensor_test = torch.tensor(matching_msi_test, dtype=torch.float32)
coords_tensor_test = torch.tensor(coords_rna_test, dtype=torch.float32)

# Define k-nearest neighbors for test set
knn = NearestNeighbors(n_neighbors=5).fit(coords_rna_test)
distances, indices = knn.kneighbors(coords_rna_test)

test_edges = []
for i, neighbors in enumerate(indices):
    for neighbor in neighbors:
        if i != neighbor:
            test_edges.append([i, neighbor])

test_edge_index = torch.tensor(test_edges, dtype=torch.long).t().contiguous()

test_data = Data(x=X_tensor_test, edge_index=test_edge_index)

# Evaluate model
model.eval()
Y_pred_train = model(train_data.x, train_data.edge_index).detach().numpy()
Y_pred_test = model(test_data.x, test_data.edge_index).detach().numpy()

